### EGOR ZAITSEV - 07 452 541

### XIAOXU YAO - 07 498 897

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import fmin

### Exercise 1: Consumption Savings Problem

Consider the following life-cycle Problem

$$\begin{aligned}&\max_{\{c_t\}_{t=0}^{T}}\sum_{t=0}^{T}\beta^tu(c_t)\\\text{s.t.}&u(c_t)=\frac{c_t^{1-\theta}-1}{1-\theta}\\&c_t+a_{t+1}=a_t(1+r)+w_t\\&a_0\text{ given and }a_{T+1}\ge0\end{aligned}$$

### Characterize the analytical solution for any $\beta,r,w_t$

#### Euler equation
Start off by writing out the objective up to the second element

$$\max_{\{c_t\}_{t=0}^{T}}u(c_0)+\beta u(c_1)+\dots$$

At the optimum, for any two arbitrary periods it must hold that

$$\frac{\partial u(\cdot)}{\partial c_0}=\beta\frac{\partial u(\cdot)}{\partial c_1}\frac{\partial c_1}{\partial c_0}$$

Using the budget constraint we may find that $\frac{\partial c_1}{\partial c_0}=1+r$ hence the Euler equation for period $t$ reads

$$c_t^{-\theta}=\beta(1+r)c_{t+1}^{-\theta}\iff\frac{c_{t+1}}{c_t}=[\beta(1+r)]^{\frac{1}{\theta}}$$

Hence, the consumption path is given by

$$c_{t+1}=[\beta(1+r)]^{\frac{1}{\theta}}c_t$$



### Compute the numerical solutions setting $w_0=10, a_0=0$ and $w_t=0$ for $t\ge1, \beta=0.99,r=0.05$ and any $\theta$

In [ ]:
# PARAMETERS
theta = 3
beta = 0.99
r = 0.05

# LIFE LENGTH
T = 10

# ENDOWMENT
w = [0.0] * T
w[0] = 10.
w = np.array(w)

# SAVINGS
a = [5] * (T-1)
a = a + [0]
a = np.array(a)

In [2]:
def euler(beta, r, theta):
    return (beta+beta*r)**(1/theta)

In [3]:
def crra(c, theta, tol=1e-5):
    """
    CRRA utility function (constant risk aversion).

    Arguments:
        c - consumption vector
        theta - risk aversion parameter
        tol - death tolerance

    Returns:
        function value
    """
    if theta != 0:
        if c < tol:
            u = -tol**(-1)
        else:
            u = (c**(1-theta) - 1)/(1-theta)
    else:
        return 'theta cannot be zero'
    
    return u


In [4]:
def budgetConstraint(a, w, r, T):
    """
    Budget constraint solved for consumption.

    Arguments:
        a - savings vector
        w - income vector
        r - interest rate
        T - agent's life length

    Returns:
        consumption vector assuming the constraint is binding
    """

    c = [0] * (T)
    c[0] = w[0] - a[0]
    for t in range(1, T-1):
        c[t] = w[t] + (1+r)*a[t-1] - a[t]
    c[T-1] = w[T-1] + (1+r)*a[T-2]

    return np.array(c)


In [5]:
def bellman(a, w, r, T, budgetConstraint, u, theta, beta):
    """
    Backward induction on the Bellman equation. Finite-horizon problems are solved via backwardation.

    Arguments:
        a - savings vector
        w - income vector
        r - interest rate
        T - agent's life length
        budgetConstraint - transform function
        u - utility function
        theta - relative risk aversion parameter
        beta - time preference parameter
    Returns:
        function value
    """
    c = budgetConstraint(a, w, r, T)
    v = u(c[-1], theta)

    for period in reversed(range(len(c) - 1)):
        v = u(c[period], theta) + beta*v
    
    return -v # return negative Bellman because we use minimizers

In [49]:
a1 = fmin(bellman, x0=a, args=(w, r, T, budgetConstraint, crra, theta, beta), maxiter=10e4)
c1 = budgetConstraint(a1, w, r, T)

Optimization terminated successfully.
         Current function value: -1.650965
         Iterations: 1417
         Function evaluations: 2022


In [50]:
data = {'w': w, 'a0': a, 'a*': a1, 'c*': c1}
df = pd.DataFrame(data)
print('numerical solution for T=10 and theta=3\n{}'.format(df.round(2)))

numerical solution for T=10 and theta=3
      w  a0    a*    c*
0  10.0   5  8.83  1.17
1   0.0   5  8.09  1.18
2   0.0   5  7.29  1.20
3   0.0   5  6.44  1.22
4   0.0   5  5.53  1.23
5   0.0   5  4.56  1.25
6   0.0   5  3.53  1.26
7   0.0   5  2.43  1.28
8   0.0   5  1.25  1.30
9   0.0   0 -0.06  1.31


To solve the Bellman equation we use simplex method (aka Nelder-Mead algorithm) which helps us minimize the objective without specifying neither first nor second derivative of the objective

Note that the vectors $\mathbf a_0$ and $\mathbf a^*$ are shifted in time by one increment. Using such a structure for the budget constraint we impose $a_0=0$ and $a_{T+1}\ge0$. We see that $a^*_{T+1}$ is slightly below zero which is a sign of truncation error of this numerical method